# Increasing the Notebook Size

In [9]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
# Importing Native Python Libraries
import os
import math
import random
import argparse
import collections
import re
from collections import namedtuple, deque, defaultdict
from itertools import count

# Importing Numpy and Pandas
import numpy as np
import pandas as pd
# import modin.pandas as pd


import re

# Importing Matplotlib
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

# Importing TA-Lib
import talib

# import PyTorch LIbraries
import torch
import torch.utils.data as utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

# Importing SciKit Learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale

# Importing RL Libraries
import gymnasium as gym

# Importing TQDM to track cell progress
from tqdm import tqdm

# Ignoring Warnings
# import warnings

# warnings.filterwarnings('ignore')

In [11]:
# Importing Custom Code
from MainCode.DataReader import NumpyReader, TimeStep, SimpleTokenizer
import MainCode.env

make sure to add .gitignore to temp_directory
file already exist


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

cuda
True
1
0
NVIDIA GeForce RTX 4090


# This Code is Only Used for Demonstration Purposes

Initially we attempted to use pytorch file type, .pt, to save our data, but it was inefficient and slow for our large data set so we switched to .npz\
However, it was sufficient for our target and index tuples so we did use it for that

In [13]:
# torch.save(dummy, 'test.pt')
# target = torch.load('target.pt')
# index = torch.load('index.pt')


# target = target.cpu().numpy()
# index = index.cpu().numpy()

# data = torch.load('main_tensor.pt').cpu().numpy()
# data

# Initializing Class to Read Our Compressed Numpy Matrices to Read as One 

To help with data storage, read/write speed, and GitHub limitations we stored our dataset in Compressed Numpy Matrix Files and the NumpyReader contains the methods and attributes needed to work on them

In [14]:
nr = NumpyReader()

In [15]:
# Uncomment to visualize the matrix
# nr.matrix

In [16]:
%%time
main_data_list = ['./datafiles/main_matrix1.npz', './datafiles/main_matrix2.npz', './datafiles/main_matrix3.npz'] # Loading our .npz files that contain our data

CPU times: total: 0 ns
Wall time: 0 ns


# Creating Matrices to hold our data to save in numpy .npz files

In [17]:
# Matrix file sizes were based on GitHub file size limits
# matrix1 = data[:20741]
# matrix2 = data[20741 : 20741 * 2]
# matrix3 = data[20741 * 2 : 20741 * 3]

In [18]:
nr.matrix.shape

(62223, 1553, 17)

# Saving our compressed data

In [19]:
# np.savez_compressed('main_matrix1.npz', matrix1)
# np.savez_compressed('main_matrix2.npz', matrix2)
# np.savez_compressed('main_matrix3.npz', matrix3)

In [20]:
env = TimeStep()
tokenizer = SimpleTokenizer()
df = env.base_data.copy()

temp = df.copy()
df = df.drop(['target'],axis=1)

In [21]:
targets = temp['target'].values

In [22]:
target_array = []
for i in df.values:
    result = " ".join(i)
    target_array.append(result)

# Tokenizing our "Words" from our Network Activity Log

In [15]:
tokens = []
for i in target_array:
    tok = re.findall(r'\w+|\S', i)
    tokens.append(tok)

df['tokens'] = tokens

In [16]:
df['new_tokens'] = df['tokens'].apply(tokenizer.convert_tokens_to_ids)

# Padding our Tokenized Matrices to Pass to Our Model

In [17]:
padded_tokens = []
for i in df['new_tokens'].values:
    padded_array = np.zeros(17)
    length_int = len(i)
    padded_array[:length_int] = i
    padded_tokens.append(padded_array)

# Creating a Feather File after Tokenization

In [18]:
df['total_padded_tokens'] = padded_tokens
df

,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,tokens,new_tokens,total_padded_tokens
Time Stamp,,,,,,,,,,,
715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,"[ANONYMOUS, LOGON, @, C457, ANONYMOUS, LOGON, ...","[1, 2, 3, 4, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8]","[1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 4.0, 4.0, ..."
715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,"[ANONYMOUS, LOGON, @, C467, ANONYMOUS, LOGON, ...","[1, 2, 3, 9, 1, 2, 3, 9, 9, 9, 5, 6, 7, 8]","[1.0, 2.0, 3.0, 9.0, 1.0, 2.0, 3.0, 9.0, 9.0, ..."
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,"[ANONYMOUS, LOGON, @, C586, ANONYMOUS, LOGON, ...","[1, 2, 3, 10, 1, 2, 3, 10, 10, 10, 5, 6, 7, 8]","[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 10...."
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success,"[ANONYMOUS, LOGON, @, C586, ANONYMOUS, LOGON, ...","[1, 2, 3, 10, 1, 2, 3, 10, 11, 10, 12, 6, 13, 8]","[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 11...."
715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success,"[ANONYMOUS, LOGON, @, C586, ANONYMOUS, LOGON, ...","[1, 2, 3, 10, 1, 2, 3, 10, 14, 10, 12, 6, 13, 8]","[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 14...."
...,...,...,...,...,...,...,...,...,...,...,...
777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success,"[U8857, @, DOM1, U8857, @, DOM1, C528, C528, ?...","[12360, 3, 21, 12360, 3, 21, 53, 53, 5, 6, 7, 8]","[12360.0, 3.0, 21.0, 12360.0, 3.0, 21.0, 53.0,..."
777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success,"[U953, @, DOM1, U953, @, DOM1, C803, C1065, Ke...","[3410, 3, 21, 3410, 3, 21, 3411, 30, 22, 6, 13...","[3410.0, 3.0, 21.0, 3410.0, 3.0, 21.0, 3411.0,..."
777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success,"[U9861, @, DOM1, U9861, @, DOM1, C457, C457, ?...","[4099, 3, 21, 4099, 3, 21, 4, 4, 5, 6, 7, 8]","[4099.0, 3.0, 21.0, 4099.0, 3.0, 21.0, 4.0, 4...."


# Final DataFrame with Time Stamps and Associated Tensors

In [19]:
df_tpt = df[['total_padded_tokens']]
df_tpt

,total_padded_tokens
Time Stamp,
715589,"[1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 4.0, 4.0, ..."
715589,"[1.0, 2.0, 3.0, 9.0, 1.0, 2.0, 3.0, 9.0, 9.0, ..."
715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 10...."
715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 11...."
715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 14...."
...,...
777813,"[12360.0, 3.0, 21.0, 12360.0, 3.0, 21.0, 53.0,..."
777813,"[3410.0, 3.0, 21.0, 3410.0, 3.0, 21.0, 3411.0,..."
777813,"[4099.0, 3.0, 21.0, 4099.0, 3.0, 21.0, 4.0, 4...."


# Scrap Work

Below is a comparison of the difference between saving and loading using a CSV and feather file

In [20]:
%%time
df_tpt.to_csv('./data_files/df_padded_tokens.csv')

CPU times: total: 14min 29s
Wall time: 14min 41s


In [22]:
%%time
df_tpt.reset_index().to_feather('./data_files/df_padded_tokens.feather')

CPU times: total: 5.23 s
Wall time: 6.18 s


In [8]:
%%time
temp = pd.read_csv('./data_files/df_padded_tokens.csv')
temp

CPU times: total: 11.7 s
Wall time: 11.9 s


,Time Stamp,total_padded_tokens
0,715589,[1. 2. 3. 4. 1. 2. 3. 4. 4. 4. 5. 6. 7. 8. 0. ...
1,715589,[1. 2. 3. 9. 1. 2. 3. 9. 9. 9. 5. 6. 7. 8. 0. ...
2,715589,[ 1. 2. 3. 10. 1. 2. 3. 10. 10. 10. 5. ...
3,715589,[ 1. 2. 3. 10. 1. 2. 3. 10. 11. 10. 12. ...
4,715589,[ 1. 2. 3. 10. 1. 2. 3. 10. 14. 10. 12. ...
...,...,...
14863570,777813,[1.236e+04 3.000e+00 2.100e+01 1.236e+04 3.000...
14863571,777813,[3.410e+03 3.000e+00 2.100e+01 3.410e+03 3.000...
14863572,777813,[4.099e+03 3.000e+00 2.100e+01 4.099e+03 3.000...
14863573,777813,[430. 3. 21. 430. 3. 21. 324. 324. 5. ...


In [7]:
%%time
temp = pd.read_feather('./data_files/df_padded_tokens.feather')
temp

CPU times: total: 3.72 s
Wall time: 4.68 s


,Time Stamp,total_padded_tokens
0,715589,"[1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 4.0, 4.0, ..."
1,715589,"[1.0, 2.0, 3.0, 9.0, 1.0, 2.0, 3.0, 9.0, 9.0, ..."
2,715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 10...."
3,715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 11...."
4,715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 14...."
...,...,...
14863570,777813,"[12360.0, 3.0, 21.0, 12360.0, 3.0, 21.0, 53.0,..."
14863571,777813,"[3410.0, 3.0, 21.0, 3410.0, 3.0, 21.0, 3411.0,..."
14863572,777813,"[4099.0, 3.0, 21.0, 4099.0, 3.0, 21.0, 4.0, 4...."
14863573,777813,"[430.0, 3.0, 21.0, 430.0, 3.0, 21.0, 324.0, 32..."


In [33]:
temp.set_index('Time Stamp', inplace=True)
temp

,total_padded_tokens
Time Stamp,
715589,"[1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 4.0, 4.0, ..."
715589,"[1.0, 2.0, 3.0, 9.0, 1.0, 2.0, 3.0, 9.0, 9.0, ..."
715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 10...."
715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 11...."
715589,"[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 14...."
...,...
777813,"[12360.0, 3.0, 21.0, 12360.0, 3.0, 21.0, 53.0,..."
777813,"[3410.0, 3.0, 21.0, 3410.0, 3.0, 21.0, 3411.0,..."
777813,"[4099.0, 3.0, 21.0, 4099.0, 3.0, 21.0, 4.0, 4...."


In [35]:
df_temp = df.reset_index()
# df_temp.to_feather('./data_files/delete_before_GitHub.feather')
df_temp 

,Time Stamp,Source_Dome,Dest_Dome,Source_PC,Destination_PC,auth_type,logon_type,Auth Orientation,result,tokens,new_tokens,total_padded_tokens
0,715589,ANONYMOUS LOGON@C457,ANONYMOUS LOGON@C457,C457,C457,?,Network,LogOff,Success,"[ANONYMOUS, LOGON, @, C457, ANONYMOUS, LOGON, ...","[1, 2, 3, 4, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8]","[1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 4.0, 4.0, ..."
1,715589,ANONYMOUS LOGON@C467,ANONYMOUS LOGON@C467,C467,C467,?,Network,LogOff,Success,"[ANONYMOUS, LOGON, @, C467, ANONYMOUS, LOGON, ...","[1, 2, 3, 9, 1, 2, 3, 9, 9, 9, 5, 6, 7, 8]","[1.0, 2.0, 3.0, 9.0, 1.0, 2.0, 3.0, 9.0, 9.0, ..."
2,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success,"[ANONYMOUS, LOGON, @, C586, ANONYMOUS, LOGON, ...","[1, 2, 3, 10, 1, 2, 3, 10, 10, 10, 5, 6, 7, 8]","[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 10...."
3,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C5995,C586,NTLM,Network,LogOn,Success,"[ANONYMOUS, LOGON, @, C586, ANONYMOUS, LOGON, ...","[1, 2, 3, 10, 1, 2, 3, 10, 11, 10, 12, 6, 13, 8]","[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 11...."
4,715589,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C6931,C586,NTLM,Network,LogOn,Success,"[ANONYMOUS, LOGON, @, C586, ANONYMOUS, LOGON, ...","[1, 2, 3, 10, 1, 2, 3, 10, 14, 10, 12, 6, 13, 8]","[1.0, 2.0, 3.0, 10.0, 1.0, 2.0, 3.0, 10.0, 14...."
...,...,...,...,...,...,...,...,...,...,...,...,...
14863570,777813,U8857@DOM1,U8857@DOM1,C528,C528,?,Network,LogOff,Success,"[U8857, @, DOM1, U8857, @, DOM1, C528, C528, ?...","[12360, 3, 21, 12360, 3, 21, 53, 53, 5, 6, 7, 8]","[12360.0, 3.0, 21.0, 12360.0, 3.0, 21.0, 53.0,..."
14863571,777813,U953@DOM1,U953@DOM1,C803,C1065,Kerberos,Network,LogOn,Success,"[U953, @, DOM1, U953, @, DOM1, C803, C1065, Ke...","[3410, 3, 21, 3410, 3, 21, 3411, 30, 22, 6, 13...","[3410.0, 3.0, 21.0, 3410.0, 3.0, 21.0, 3411.0,..."
14863572,777813,U9861@DOM1,U9861@DOM1,C457,C457,?,Network,LogOff,Success,"[U9861, @, DOM1, U9861, @, DOM1, C457, C457, ?...","[4099, 3, 21, 4099, 3, 21, 4, 4, 5, 6, 7, 8]","[4099.0, 3.0, 21.0, 4099.0, 3.0, 21.0, 4.0, 4...."
14863573,777813,U996@DOM1,U996@DOM1,C10,C10,?,Network,LogOff,Success,"[U996, @, DOM1, U996, @, DOM1, C10, C10, ?, Ne...","[430, 3, 21, 430, 3, 21, 324, 324, 5, 6, 7, 8]","[430.0, 3.0, 21.0, 430.0, 3.0, 21.0, 324.0, 32..."


In [25]:
# temp = pd.read_feather('./data_files/delete_before_GitHub.feather')

In [26]:
# df = temp[['total_padded_tokens']]
# del temp

In [ ]:
data = df['total_padded_tokens'].values

In [ ]:
observation_data = np.vstack(data)

In [ ]:
#and this cell
matrices_list = []
env.reset()
window_size = env.index_list[env.current_step]
# window = env.data[window_size[0]:window_size[1]]

for i in env.index_list[:-1]:
    
    observation = observation_data[i[0] : i[1]]
    matrix = np.zeros((1553, 17))

    
    matrix[:observation.shape[0], :] = observation
    env.array_step()
    matrices_list.append(matrix)
    window_size = env.index_list[env.current_step]

In [ ]:
larger_list = np.array(matrices_list)

In [ ]:
login_tensors = torch.tensor(larger_list, dtype=torch.float32)
login_tensors